Azure offers tools such as Azure Machine Learning that facilitate the deployment and scaling of machine learning models, including neural networks.

A neural network is composed of layers of neurons, where each neuron processes inputs and passes the output to the next layer. In this guide, we will focus on building and training a simple feedforward neural network using the Azure Machine Learning SDK and PyTorch, a widely used deep learning library.

By the end of this lesson, you will be able to:

Describe the steps needed for the basic implementation of a neural network using Python and Microsoft Azure. 

In [2]:
from azureml.core import Workspace

# Create or retrieve an existing Azure ML workspace
ws = Workspace.get(name='Phuonghqh1',
                      subscription_id='9f6f29a3-541d-4d32-93a4-9004b107807d',
                      resource_group='test1',
                      location='Southeast Asia')

#ws = Workspace.create(name='myworkspace',
#                      subscription_id='your-subscription-id',
#                      resource_group='myresourcegroup',
#                      location='eastus')

# Write configuration to the workspace config file
ws.write_config(path='.azureml')

ModuleNotFoundError: No module named 'azureml'

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network with one hidden layer
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Input layer (784 input features)
        self.fc2 = nn.Linear(128, 10)   # Output layer (10 classes)
        self.relu = nn.ReLU()           # Activation function

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Apply ReLU after the first layer
        x = self.fc2(x)             # Output layer
        return x

# Initialize the neural network
model = SimpleNN()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # For classification tasks
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Stochastic Gradient Descent

In [5]:
from torchvision import datasets, transforms

# Load MNIST dataset
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=32, shuffle=True)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Reset gradients

        # Forward pass
        outputs = model(inputs.view(-1, 784))  # Flatten input
        loss = criterion(outputs, labels)      # Compute loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

100.0%
100.0%
100.0%
100.0%


Epoch 1, Loss: 0.8687211656411489
Epoch 2, Loss: 0.3693331449667613
Epoch 3, Loss: 0.314665305797259
Epoch 4, Loss: 0.2832479240675767
Epoch 5, Loss: 0.259672743088007


In [6]:
from azureml.core import Model

# Save the trained model
torch.save(model.state_dict(), 'simple_nn.pth')

# Register the model in Azure
model = Model.register(workspace=ws, model_path='simple_nn.pth', model_name='simple_nn')

# Deploying as a web service requires more steps such as creating a scoring script and environment.

Registering model simple_nn
